# Extract the Travel Time for the Locations of Different Properties

## Import Packages

In [19]:
# Web - Scraping and API Requests
import requests
from httpx import AsyncClient, Response
from parsel import Selector
import parsel
import jmespath
import asyncio

# Data Manipulation and Analysis
import pandas as pd
from pprint import pprint 
import json
from typing import List
from typing import TypedDict

# Database Connection
from sqlalchemy import create_engine

# File and System Operations
import os
import sys

In [20]:
pd.set_option('display.max_columns', None) # Display all columns in any given DataFrame

## Other Setup

### Finding the correctd file directory for the credentials json with the api key

In [21]:
current_dir = os.path.dirname(os.path.abspath("NB03_Extract_Travel_Time_Data.ipynb"))
sys.path.insert(0,os.path.join(current_dir, '..'))

credentials_file_path = os.path.join(current_dir, '..', "credentials.json")

# open the file and load the data into a variable
with open(credentials_file_path, "r") as f:
    credentials = json.load(f)

In [22]:
pprint(credentials)

{'api_key': '716417c04acf3944937af85d78775ea0', 'app_id': '27beab81'}


In [44]:
# Input the credentials into the header
headers = {
    "Content-Type": "application/json",
    "X-Application-Id": credentials["app_id"],
    "X-Api-Key": credentials["api_key"]
}

# The payload from the curl request
payload = {
    "arrival_searches": {
        "one_to_many": [
            {
                "id": "Example Search",
                "departure_location_id": "Origin",
                "arrival_location_ids": [
                    "Destination 1", "Destination 2", "Destination 3"
                ],
                "transportation": {
                    "type": "public_transport"
                },
                "travel_time": 10800,
                "arrival_time_period": "weekday_morning",
                "properties": ["travel_time", "distance"]
            }
        ]
    },
    "locations": [
        {
            "id": "Origin", # make this the co-ordinates of bank station
            "coords": {
                "lat": 51.513,
                "lng": -0.088
            }
        },
        {
            "id": "Destination 1", # make this oxford circus station
            "coords": {
                "lat": 51.5152,
                "lng": -0.1419
            }
        },
        {
            "id": "Destination 2", # make this some property in walthamstowe
            "coords": {
                "lat": 51.576702,
                "lng": -0.02847
            }
        },
        {
            "id": "Destination 3", # make this a property in nine elms
            "coords": {
                "lat": 51.5163,
                "lng": -0.1300	
            }
        }
    ]
}

# Make the request
response = requests.post(
    "https://api.traveltimeapp.com/v4/time-filter/fast",
    headers=headers,
    data=json.dumps(payload)
)

# Check the response
print(response.status_code)
pprint(response.json())  # or response.text if not JSON

200
{'results': [{'locations': [{'id': 'Destination 1',
                             'properties': {'distance': 0, 'travel_time': 908}},
                            {'id': 'Destination 2',
                             'properties': {'distance': 0,
                                            'travel_time': 2199}},
                            {'id': 'Destination 3',
                             'properties': {'distance': 0,
                                            'travel_time': 857}}],
              'search_id': 'Example Search',
              'unreachable': []}]}


In [41]:
headers = {
    "Content-Type": "application/json",
    "X-Application-Id": credentials["app_id"],
    "X-Api-Key": credentials["api_key"]
}

payload = {
    "locations": [
        {
            "id": "starting-location",
            "coords": {"lng": -0.0866253, "lat": 51.5140401}
        },
        {
            "id": "other-location-0",
            "coords": {"lat": 51.51126100696778, "lng": -0.07532395813711054}
        },
        {
            "id": "other-location-1",
            "coords": {"lat": 51.52642194622867, "lng": -0.09520781476778037}
        },
        {
            "id": "other-location-2",
            "coords": {"lat": 51.49938271146974, "lng": -0.07804923576430915}
        },
        {
            "id": "other-location-3",
            "coords": {"lat": 51.50884919364359, "lng": -0.07209628542344454}
        },
        {
            "id": "other-location-4",
            "coords": {"lat": 51.5153, "lng": -0.142}
        }
    ],
    "arrival_searches": {
        "one_to_many": [
            {
                "id": "arrive-at one-to-many search example",
                "arrival_location_ids": [
                    "other-location-0",
                    "other-location-1",
                    "other-location-2",
                    "other-location-3",
                    "other-location-4"
                ],
                "departure_location_id": "starting-location",
                "arrival_time_period": "weekday_morning",
                "properties": ["travel_time"],
                "transportation": {"type": "public_transport"},
                "travel_time": 1800
            }
        ],
        "many_to_one": []
    }
}

response = requests.post(
    "https://api.traveltimeapp.com/v4/time-filter/fast",
    headers=headers,
    json=payload
)

print(response.status_code)
pprint(response.json())

200
{'results': [{'locations': [{'id': 'other-location-3',
                             'properties': {'travel_time': 698}},
                            {'id': 'other-location-2',
                             'properties': {'travel_time': 1114}},
                            {'id': 'other-location-1',
                             'properties': {'travel_time': 1043}},
                            {'id': 'other-location-0',
                             'properties': {'travel_time': 563}},
                            {'id': 'other-location-4',
                             'properties': {'travel_time': 978}}],
              'search_id': 'arrive-at one-to-many search example',
              'unreachable': []}]}
